In [1]:
import numpy as np
np.median(np.array([[0.00299,2.06,125.0,0.0177,10.4,122.0,0.0093,10.9,84.1],[0.00631,2.53,125.0,0.00844,4.67,126.0,0.0103,6.3,80.9],[0.00465,3.92,109.0,0.0136,12.0,124.0,0.00498,2.13,79.6]]), axis=0)

array([4.65e-03, 2.53e+00, 1.25e+02, 1.36e-02, 1.04e+01, 1.24e+02,
       9.30e-03, 6.30e+00, 8.09e+01])

### Testing data cleaning codes on one json item

In [2]:
# import json
# import numpy as np
# import pandas as pd
# with open('data_test.json') as train_file:
#     data = json.load(train_file)

In [3]:
# transcript_name = []
# position_num = []
# bases_corresponding_to_position = []
# df = pd.DataFrame(columns=['Transcript Name', 'Position', 'Bases', 'SD', 'Mean', 'Dwelling Time'])
# for k, v in data.items(): # obtain transcript name and rest of the items
#     print(k)
#     print(v)
#     transcript_name.append(k)
#     transcript_name = transcript_name * 3
#     transcript_name = pd.DataFrame(transcript_name, columns=['Transcript Name'])
#     print('')
#     for i,j in v.items(): # obtain position and rest of items
#         print(i)
#         print(j)
#         position_num.append(int(i) - 1)
#         position_num.append(int(i))
#         position_num.append(int(i) + 1)
#         position_num = pd.DataFrame(position_num, columns = ['Position'])
#         print('')
#         for a,b in j.items(): # obtain bases and measurements - dwell time, etc.
#             print("a", a)
#             print("b", b)
#             bases_corresponding_to_position.append(a[:5])
#             bases_corresponding_to_position.append(a[1:6])
#             bases_corresponding_to_position.append(a[2:])
#             bases_corresponding_to_position = pd.DataFrame(bases_corresponding_to_position, columns = ['Bases'])
            
#             arr = np.array(b)
#             mean_measurement_res = np.mean(arr, axis=0)
#             print(mean_measurement_res)
#             measurement_first_pos = mean_measurement_res[0:3]
#             measurement_second_pos = mean_measurement_res[3:6]
#             measurement_third_pos = mean_measurement_res[6:]
            
#             measurement_first_pos = measurement_first_pos.reshape(1,3)
#             measurement_second_pos = measurement_second_pos.reshape(1,3)
#             measurement_third_pos = measurement_third_pos.reshape(1,3)
#             measurement_first_pos = pd.DataFrame.from_records(measurement_first_pos, columns = ['Dwelling Time', 'SD', 'Mean'])
#             measurement_second_pos = pd.DataFrame.from_records(measurement_second_pos, columns = ['Dwelling Time', 'SD', 'Mean'])
#             measurement_third_pos = pd.DataFrame.from_records(measurement_third_pos, columns = ['Dwelling Time', 'SD', 'Mean'])
            
#             all_measurement = pd.concat([measurement_first_pos, measurement_second_pos, measurement_third_pos], axis=0).reset_index(drop=True)
#             full_df = pd.concat([transcript_name, position_num, bases_corresponding_to_position, all_measurement], axis=1)
            
#             print(full_df)

### Data cleaning codes for all the json items

Read measurements are given in the following form: `[0.00299,2.06,125.0,0.0177,10.4,122.0,0.0093,10.9,84.1]`. We can interpret the numbers as such:
- 1st, 4th and 7th item refers to the dwelling time of the 5-mer nucleotides
- 2nd, 5th and 8th item refers to the standard deviation of the direct RNA current 
- 3rd, 6th and 9th item refers to the mean of the direct RNA current

For each read we have extracted features from segments corresponding to 1-neighboring position and this results in a total of 9 features for each read (3 features for position 243, followed by 3 features for positions 244 and 245 respectively).

Since there are variable number of reads for each position, will be trying to obtain the **mean/ median** values across all the reads. E.g., if the measurements are `[0.00299,2.06,125.0,0.0177,10.4,122.0,0.0093,10.9,84.1],[0.00631,2.53,125.0,0.00844,4.67,126.0,0.0103,6.3,80.9],[0.00465,3.92,109.0,0.0136,12.0,124.0,0.00498,2.13,79.6]`, the result I will be using is `[4.65e-03, 2.53e+00, 1.25e+02, 1.36e-02, 1.04e+01, 1.24e+02, 9.30e-03, 6.30e+00, 8.09e+01]`

In [4]:
import json
import numpy as np
import pandas as pd
data_list = [json.loads(line)
        for line in open('aws_data.json', 'r', encoding='utf-8')]

In [5]:
len(data_list)

1008933

In [ ]:
counter=1
first = 'aws_full_df_'
last = '.csv'

df = pd.DataFrame(columns=['Transcript Name', 'Position', 'Bases', 'Mean SD', 'Mean_Mean', 'Mean Dwelling Time', 'Median SD', 'Median_Mean', 'Median Dwelling Time'])
for data in data_list:
    transcript_name = []
    position_num = []
    bases_corresponding_to_position = []
    for k, v in data.items(): # obtain transcript name and rest of the items
        transcript_name.append(k)
        transcript_name = transcript_name * 3
        transcript_name = pd.DataFrame(transcript_name, columns=['Transcript Name'])
        for i,j in v.items(): # obtain position and rest of items
            position_num.append(int(i) - 1)
            position_num.append(int(i))
            position_num.append(int(i) + 1)
            position_num = pd.DataFrame(position_num, columns = ['Position'])
            for a,b in j.items(): # obtain bases and measurements - dwell time, etc.
                bases_corresponding_to_position.append(a[:5])
                bases_corresponding_to_position.append(a[1:6])
                bases_corresponding_to_position.append(a[2:])
                bases_corresponding_to_position = pd.DataFrame(bases_corresponding_to_position, columns = ['Bases'])

                arr = np.array(b)
                
                # For median readings
                median_measurement_res = np.median(arr, axis=0)
                median_measurement_first_pos = median_measurement_res[0:3]
                median_measurement_second_pos = median_measurement_res[3:6]
                median_measurement_third_pos = median_measurement_res[6:]

                median_measurement_first_pos = median_measurement_first_pos.reshape(1,3)
                median_measurement_second_pos = median_measurement_second_pos.reshape(1,3)
                median_measurement_third_pos = median_measurement_third_pos.reshape(1,3)
                median_measurement_first_pos = pd.DataFrame.from_records(median_measurement_first_pos, columns = ['Median Dwelling Time', 'Median SD', 'Median_Mean'])
                median_measurement_second_pos = pd.DataFrame.from_records(median_measurement_second_pos, columns = ['Median Dwelling Time', 'Median SD', 'Median_Mean'])
                median_measurement_third_pos = pd.DataFrame.from_records(median_measurement_third_pos, columns = ['Median Dwelling Time', 'Median SD', 'Median_Mean'])

                all_median_measurement = pd.concat([median_measurement_first_pos, median_measurement_second_pos, median_measurement_third_pos], axis=0).reset_index(drop=True)
                
                # For mean readings
                mean_measurement_res = np.mean(arr, axis=0)
                measurement_first_pos = mean_measurement_res[0:3]
                measurement_second_pos = mean_measurement_res[3:6]
                measurement_third_pos = mean_measurement_res[6:]

                measurement_first_pos = measurement_first_pos.reshape(1,3)
                measurement_second_pos = measurement_second_pos.reshape(1,3)
                measurement_third_pos = measurement_third_pos.reshape(1,3)
                measurement_first_pos = pd.DataFrame.from_records(measurement_first_pos, columns = ['Mean Dwelling Time', 'Mean SD', 'Mean_Mean'])
                measurement_second_pos = pd.DataFrame.from_records(measurement_second_pos, columns = ['Mean Dwelling Time', 'Mean SD', 'Mean_Mean'])
                measurement_third_pos = pd.DataFrame.from_records(measurement_third_pos, columns = ['Mean Dwelling Time', 'Mean SD', 'Mean_Mean'])

                all_measurement = pd.concat([measurement_first_pos, measurement_second_pos, measurement_third_pos], axis=0).reset_index(drop=True)
                
                full_df = pd.concat([transcript_name, position_num, bases_corresponding_to_position, all_measurement, all_median_measurement], axis=1)

                df = df.append(full_df).reset_index(drop=True)
                
                # codes to make concatenation more efficient
                if len(df) == 100002:
                    print(len(df))
                    
                    directory = first+str(counter)+last
                    df.to_csv(directory)
                    counter+=1
#                     print(counter)
#                     print(directory)
                    
                    # deleting df so that the concatenation time will be shorter
                    del df
                    df = pd.DataFrame(columns=['Transcript Name', 'Position', 'Bases', 'Mean SD', 'Mean_Mean', 'Mean Dwelling Time', 'Median SD', 'Median_Mean', 'Median Dwelling Time'])

# For the final subset of data that is <= 100002 rows  
directory = first+str(counter)+last
df.to_csv(directory)

In [ ]:
df

In [ ]:
df.info()

In [ ]:
full_df.info()